# 可迭代对象、迭代器和生成器

> Every generator is an iterator: generators fully implement the iterator interface. But an iterator—as defined in the GoF book— retrieves items from a collection, while a generator can produce items “out of thin air.” That’s why the Fibonacci sequence gener‐ ator is a common example: an infinite series of numbers cannot be stored in a collection.  -- *Fluent Pyton*

《流畅的Python》这本书中告诉我们：每个生成器都是一个迭代器。为什么这么说？因为生成器完全实现了迭代器的接口。但是一个迭代器需要从集合中取出元素，而生成器却可以无中生有。

## 1. 可迭代对象

### 1.1 实例：单词序列

我们通过构造一个Sentence类，来打开探索可迭代对象的旅程。我们传入一些文本的字符串，然后可以逐个单词迭代。

In [1]:
import re
import reprlib


RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __getitem__(self, index):
        return self.words[index]
    
    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence({})'.format(reprlib.repr(self.text))

In [2]:
s = Sentence('"The time has come," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [3]:
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


In [4]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

In [5]:
from collections.abc import Iterable

myText = "The world is carzy."
sentence = Sentence(myText)

print([i for i in sentence])
print(sentence[2])
print(isinstance(sentence, Iterable))  # 没有实现 __iter__ 方法，所以是False
print(len(sentence))
print(sentence.__repr__())

['The', 'world', 'is', 'carzy']
is
False
4
Sentence('The world is carzy.')


**如果你已经理解了 re.comile 和 re.findall，以下内容可折叠。**

re.comile 用于正则匹配，传入参数为正则表达式，输出为一个模式对象（Pattern object），它常和 search, search 和 match 方法一起使用。

查看 re.py 文件，关于 compile 方法它是这么解释的：

> Compile a regular expression pattern, returning a Pattern object.

关于 findall 方法它的解释是：

> Return a list of all non-overlapping matches in the string.
> 
> If one or more capturing groups are present in the pattern, return a list of groups; this will be a list of tuples if the pattern has more than one group.
> 
> Empty matches are included in the result.

关于 search 方法它的解释是：

> Scan through string looking for a match to the pattern, returning a Match object, or None if no match was found.

关于 match 方法它的解释是：

> Try to apply the pattern **at the start of the string**, returning
a Match object, or None if no match was found.

In [6]:
# 四个方法 compile/findall/search/match 的演示
import re

myText = "The world is carzy."
WORD = re.compile('\w*r\w*')
print("WORD:", RE_WORD)

FINDALL_WORD = RE_WORD.findall(myText)
print("FINDALL_WORD:", FINDALL_WORD)

SEARCH_WORD = RE_WORD.search(myText)
print("SEARCH_WORD:", SEARCH_WORD)

MATCH_WORD = RE_WORD.match(myText)
print("MATCH_WORD:", MATCH_WORD)

WORD: re.compile('\\w+')
FINDALL_WORD: ['The', 'world', 'is', 'carzy']
SEARCH_WORD: <re.Match object; span=(0, 3), match='The'>
MATCH_WORD: <re.Match object; span=(0, 3), match='The'>


### 1.2 序列可迭代的原因：iter 函数

解释器需要迭代对象 x 时，会自动调用 iter(x)。

内置的 iter 函数有以下作用。

（1）检查对象是否实现了 \_\_iter\_\_ 方法，如果实现了就调用它，获取一个迭代器；

（2）如果没有实现 \_\_iter\_\_ 方法，但实现了 \_\_getitem\_\_ 方法，Python 会创建一个迭代器，尝试按顺序获取元素；

（3）如果尝试失败，Python 抛出TypeError 异常，通常会提示"C object is not iterable"，其中 C 是目标对象所属的类。

因此，任何 Python 序列都可迭代的原因是，它们都实现了 \_\_getitem\_\_ 方法。其实，标准的序列也都实现了 \_\_iter\_\_ 方法，因此你也应该这么做。之所以对 \_\_getitem\_\_ 方法做特殊处理，是为了向后兼容，而未来可能不会再这么做(不过，写作本书时还未弃用)。

从 Python 3.4 开始，检查对象 x 能否迭代，最准确的方法 是:调用 iter(x) 函数，如果不可迭代，再处理 TypeError 异 常。这比使用 isinstance(x, abc.Iterable) 更准确，因为 iter(x) 函数会考虑到遗留的 \_\_getitem\_\_ 方法，而abc.Iterable 类则不考虑。

iter() 函数用来生成迭代器。

In [7]:
nlist = [1,2,3]
for i in iter(nlist):
    print(i)

1
2
3


## 2. 可迭代的对象与迭代器的对比

> *iterable*
>
> Any object from which the iter built-in function can obtain an iterator. Objects implementing an \_\_iter\_\_ method returning an iterator are iterable. Sequencesare always iterable; as are objects implementing a \_\_getitem\_\_ method that takes 0-based indexes.

可迭代的对象的定义：

使用 iter 内置函数可以获取迭代器的对象。如果对象实现了能返 回迭代器的 \_\_iter\_\_ 方法，那么对象就是可迭代的。序列都可以迭代;实现了 \_\_getitem\_\_ 方法，而且其参数是从零开始的索引，这种对象也可以迭代。

可迭代的对象和迭代器之间的关系:Python 从可迭代的对象 中获取迭代器。

下面是一个简单的 for 循环，迭代一个字符串。这里，字符串 'ABC' 是可迭代的对象。背后是有迭代器的，只不过我们看不到:

In [8]:
s = 'ABC'
for char in s:
    print(char)

A
B
C


如果没有 for 语句，不得不使用 while 循环模拟，要像下面这样写:

In [9]:
s = 'ABC'
it = iter(s)  # ➊
while True:
    try:
        print(next(it))  # ➋
    except StopIteration:  # ➌ 
        del it  # ➍
        break  # ➎

A
B
C


❶ 使用可迭代的对象构建迭代器 it。

❷ 不断在迭代器上调用 next 函数，获取下一个字符。

❸ 如果没有字符了，迭代器会抛出 StopIteration 异常。

❹ 释放对 it 的引用，即废弃迭代器对象。

❺ 退出循环。

标准的迭代器接口有两个方法。

\_\_next\_\_

  返回下一个可用的元素，如果没有元素了，抛出 StopIteration 异常。

\_\_iter\_\_

  返回 self，以便在应该使用可迭代对象的地方使用迭代器，例如在 for 循环中.

下面展示如何构建和使用迭代器：

In [10]:
s3 = Sentence('Pig and Pepper')
it = iter(s3)
it

In [11]:
next(it), next(it), next(it)

('Pig', 'and', 'Pepper')

In [12]:
list(it)

[]

In [13]:
it = iter(s3)
list(it)

['Pig', 'and', 'Pepper']

In [14]:
it = iter(iter(s3))
list(it)

['Pig', 'and', 'Pepper']

> *iterator*
> 
> Any object that implements the \_\_next\_\_ no-argument method that returns the next item in a series or raises StopIteration when there are no more items. Python iterators also implement the \_\_iter\_\_ method so they are iterable as well.

迭代器的定义：

迭代器是这样的对象:实现了无参数的 \_\_next\_\_ 方法，返回序列中的下一个元素; 如果没有元素了，那么抛出 StopIteration 异常。Python 中的迭代器还实现了 \_\_iter\_\_ 方法，因此迭代器也可以迭代。

综上，我的理解是：

（1）可迭代对象和迭代器都是对象。

（2）可迭代对象在进行迭代时，可以通过 iter() 方法生成迭代器进行迭代。

（3）而迭代器通过 \_\_next\_\_ 方法实现迭代。并且，因为迭代器也实现了 \_\_iter\_\_ 方法，且该方法返回其本身。因此，迭代器也可以使用 iter() 方法进行迭代（虽然这么做没啥意义）。

对于可迭代对象和迭代器之间的关系，下面这张图的解释就很精髓。

![](./img/iter.PNG)

## 3. 典型的迭代器

下例定义的 Sentence 类可以迭代，因为它实现了特殊的 \_\_iter\_\_ 方法，构建并返回一个 SentenceIterator 实例。

In [15]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __repr__(self):
        return 'Sentence({})'.format(reprlib.repr(self.text))
    
    def __iter__(self):  # ❶
        return SentenceIterator(self.words)  # ❷
    
class SentenceIterator:
    
    def __init__(self, words):
        self.words = words  # ❸
        self.index = 0  # ❹
        
    def __next__(self):
        try:
            word = self.words[self.index]  # ❺
        except IndexError:
            raise StopIteration()  # ❻
        self.index += 1  # ❼
        return word  # ❽
    
    def __iter__(self):  # ❾
        return self

❶ 与前一版相比，这里只多了一个 __iter__ 方法。这一版没有 __getitem__ 方法，为的是明确表明这个类可以迭代，因为实现了 __iter__ 方法。

❷ 根据可迭代协议，__iter__ 方法实例化并返回一个迭代器。 

❸ SentenceIterator 实例引用单词列表。

❹ self.index 用于确定下一个要获取的单词。

❺ 获取 self.index 索引位上的单词。

❻ 如果 self.index 索引位上没有单词，那么抛出 StopIteration 异常。

❼ 递增 self.index 的值。

❽ 返回单词。

❾ 实现 self.__iter__ 方法。

In [16]:
s = Sentence('"The time has come," the Walrus said,')
it = iter(s)
while True:
    try:
        print(next(it))
    except StopIteration:
        del it
        break

The
time
has
come
the
Walrus
said


注意，对这个示例来说，其实没必要在 SentenceIterator 类中实现 \_\_iter\_\_ 方法，不过这么做是对的，因为迭代器应该实现 \_\_next\_\_ 和 \_\_iter\_\_ 两个方法，而且这么做能让迭代器通过 issubclass(SentenceInterator, abc.Iterator) 测试。如果让 SentenceIterator 类继承 abc.Iterator 类，那么它会继承  abc.Iterator.\_\_iter\_\_ 这个具体方法。

注意，SentenceIterator 类的大多数代码都在处理迭代器的内部状态。稍后会说明如何简化。不过，在此之前我们先稍微离题，讨论一个看似合理实则错误的实现捷径。

### \* 把Sentence变成迭代器:坏主意

构建可迭代的对象和迭代器时经常会出现错误，原因是混淆了二者。要知道，可迭代的对象有个 \_\_iter\_\_ 方法，每次都实例化一个新的迭 代器;而迭代器要实现 \_\_next\_\_ 方法，返回单个元素，此外还要实现 \_\_iter\_\_ 方法，返回迭代器本身。

因此，迭代器可以迭代，但是可迭代的对象不是迭代器。

除了 \_\_iter\_\_ 方法之外，你可能还想在 Sentence 类中实现
\_\_next\_\_ 方法，让 Sentence 实例既是可迭代的对象，也是自身的迭代器。可是，这种想法非常糟糕。根据有大量 Python 代码审查经验的 Alex Martelli 所说，这也是常见的反模式 (anti-pattern)。

《设计模式:可复用面向对象软件的基础》一书讲解迭代器设计模式时，在“适用性”一节中说:

迭代器模式可用来:

- 访问一个聚合对象的内容而无需暴露它的内部表示
- 支持对聚合对象的多种遍历
- 为遍历不同的聚合结构提供一个统一的接口(即支持多态迭代)

为了“支持多种遍历”，必须能从同一个可迭代的实例中获取多个独立的迭代器，而且**各个迭代器要能维护自身的内部状态**，因此这一模式正确的实现方式是，每次调用 iter(my_iterable) 都新建一个独立的迭代器。这就是为什么这个示例需要定义 SentenceIterator 类。

> **Note:** 
> 
> 可迭代的对象一定不能是自身的迭代器。也就是说，可迭代的对象必须实现 \_\_iter\_\_ 方法，但不能实现 \_\_next\_\_ 方法。
> 
> 另一方面，迭代器应该一直可以迭代。迭代器的 \_\_iter\_\_ 方法应该返回自身。

至此，我们演示了如何正确地实现典型的迭代器模式。本节至此告一段落，下一节展示如何使用更符合 Python 习惯的方式实现 Sentence 类。

## 4. Sentence 类第3版: 生成器函数

实现相同功能，但却符合 Python 习惯的方式是，用生成器函数代替
SentenceIterator 类。

先看下例，然后详细说明生成器函数。

In [17]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __repr__(self):
        return 'Sentence({})'.format(reprlib.repr(self.text))
    
    def __iter__(self):
        for word in self.words:  # ❶
            yield word  # ❷
        return  # ❸
    
# 完成！  # ❹

❶ 迭代 self.words。

❷ 产出当前的 word。

❸ 这个 return 语句不是必要的;这个函数可以直接“落空”，自动返回。不管有没有 return 语句，生成器函数都不会抛出 StopIteration 异常，而是在生成完全部值之后会直接退出。

❹ 不用再单独定义一个迭代器类!

In [18]:
s = Sentence('"The time has come," the Walrus said,')
it = iter(s)
while True:
    try:
        print(next(it))
    except StopIteration:
        del it
        break

The
time
has
come
the
Walrus
said


在上一节定义的 Sentence 类中，\_\_iter\_\_ 方法调用 SentenceIterator 类的构造方法创建一个迭代器并将其返回。而在本例中，迭代器其实是生成器对象，每次调用 \_\_iter\_\_ 方法都会自动创建，因为这里的 \_\_iter\_\_ 方法是生成器函数。

？？？为啥会自动创建？？？

下面全面说明生成器函数。

### \* 生成器函数的工作原理

只要 Python 函数的定义体中有 yield 关键字，该函数就是生成器函数。调用生成器函数时，会返回一个生成器对象。也就是说，生成器函数是生成器工厂。

> **Note:** 普通的函数与生成器函数在句法上唯一的区别是，在后者的定义体中有 yield 关键字。有些人认为定义生成器函数应该使用 一个新的关键字，例如 gen，而不该使用 def，但是 Guido 不同意。他的理由参见 [PEP 255—Simple Generators](https://www.python.org/dev/peps/pep-0255/)。

下面以一个特别简单的函数说明生成器的行为:

In [19]:
# 只要 Python 函数中包含关键字 yield，该函数就是生成器函数。
def gen_123():
    yield 1
    yield 2
    yield 3
    
# 仔细看，gen_123 是函数对象。
gen_123

<function __main__.gen_123()>

In [20]:
# 但是调用时，gen_123() 返回一个生成器对象。
gen_123()

<generator object gen_123 at 0x10b6d9d50>

In [21]:
# 生成器是迭代器，会生成传给 yield 关键字的表达式的值。
for i in gen_123():
    print(i)

1
2
3


In [22]:
# 同理
list(iter(gen_123()))

[1, 2, 3]

In [23]:
# 为了仔细检查，我们把生成器对象赋值给 g。
g = gen_123()

# 因为 g 是迭代器，所以调用 next(g) 会获取 yield 生成的下一个元素。
while True:
    try:
        print(next(g))
    except StopIteration:
        del g
        break

1
2
3


关于生成器函数的解释的英文原文：

> A generator function builds a generator object that wraps the body of the function. When we invoke next(...) on the generator object, execution advances to the next yield in the function body, and the next(...) call evaluates to the value yielded when the function body is suspended. Finally, when the function body returns, the enclosing generator object raises StopIteration, in accordance with the Iterator protocol.

翻译：

生成器函数会将生成器函数的定义体包装成一个生成器对象。把生成器传给 next(...) 函数时，生成器函数会执行函数定义体中的下一个 yield 语句，返回产出的值，并在函数定义体的当前位置暂停。最终，函数的定义体返回时，外层的生成器对象会抛出 StopIteration 异常 —— 这一点与迭代器协议一致。

> **Note:** 使用准确的词语描述从生成器中获取结果的过程，有助于理解生成器。注意，我说的是产出或生成值。如果说生成 器“返回”值，就会让人难以理解。函数返回值；调用生成器函数返回生成器；生成器产出或生成值。生成器不会以常规的方式“返回”值: 生成器函数定义体中的 return 语句会触发生成器对象抛出 StopIteration 异常。

In [24]:
def gen_AB(): # ➊ 
    print('start') 
    yield 'A' # ➋ 
    print('continue') 
    yield 'B' # ➌ 
    print('end.') # ➍

for c in gen_AB(): # ➎
    print('-->', c) # ➏

start
--> A
continue
--> B
end.


❶ 定义生成器函数的方式与普通的函数无异，只不过要使用 yield 关键字。

❷ 在 for 循环中第一次隐式调用 next() 函数时(序号➎)，会打印'start'，然后停在第一个 yield 语句，生成值 'A'。

❸ 在 for 循环中第二次隐式调用 next() 函数时，会打印'continue'，然后停在第二个 yield 语句，生成值 'B'。

❹ 第三次调用 next() 函数时，会打印 'end.'，然后到达函数定义体的末尾，导致生成器对象抛出 StopIteration 异常。

❺ 迭代时，for 机制的作用与 g = iter(gen_AB()) 一样，用于获取生成器对象，然后每次迭代时调用 next(g)。到达生成器函数定义体的末尾时，生成器对象抛出 StopIteration 异常。for 机制会捕获异常，因此循环终止时没有报错。

❻ 循环块打印 --> 和 next(g) 返回的值。但是，生成器函数中的 print 函数输出结果之后才会看到这个输出。

总结：

现在，希望你已经知道示例 14-5 中 Sentence.\_\_iter\_\_ 方法的作用了: \_\_iter\_\_ 方法是生成器函数，调用时会构建一个实现了迭代器接口的生成器对象，因此不用再定义 SentenceIterator 类了。

## 5. Sentence 类第4版: 惰性实现

设计 Iterator 接口时考虑到了惰性:next(my_iterator) 一次生成一个元素。懒惰的反义词是急迫，其实，惰性求值(lazy evaluation) 和及早求值(eager evaluation)是编程语言理论方面的技术术语。

目前实现的几版 Sentence 类都不具有惰性，因为 \_\_init\_\_ 方法急迫地构建好了文本中的单词列表，然后将其绑定到 self.words 属性上。这样就得处理整个文本，列表使用的内存量可能与文本本身一样多 (或许更多，这取决于文本中有多少非单词字符)。如果只需迭代前几个单词，大多数工作都是白费力气。

只要使用的是 Python 3，思索着做某件事有没有懒惰的方式，答案通常都是肯定的。

re.finditer 函数是 re.findall 函数的惰性版本，返回的不是列表，而是一个生成器，按需生成 re.MatchObject 实例。如果有很多匹配，re.finditer 函数能节省大量内存。我们要使用这个函数让第 4 版 Sentence 类变得懒惰，即只在需要时才生成下一个单词。代码如下所示。

In [25]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text  # ➊ 
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        for match in RE_WORD.finditer(self.text):  # ➋
            yield match.group()  # ➌

❶ 不再需要 words 列表。

❷ finditer 函数构建一个迭代器，包含 self.text 中匹配 RE_WORD 的单词，产出 MatchObject 实例。

❸ match.group() 方法从 MatchObject 实例中提取匹配正则表达式的具体文本。

In [26]:
s = Sentence('"The time has come," the Walrus said,')
it = iter(s)
while True:
    try:
        print(next(it))
    except StopIteration:
        del it
        break

The
time
has
come
the
Walrus
said


## 6. Sentence类第5版: 生成器表达式

简单的生成器函数可以替换成生成器表达式。

生成器表达式可以理解为列表推导的惰性版本: 不会迫切地构建列表，而是返回一个生成器，按需惰性生成元素。也就是说，如果列表推导是制造列表的工厂，那么生成器表达式就是制造生成器的工厂。

下例演示了一个简单的生成器表达式，并且与列表推导做了对比。

In [27]:
def gen_AB():  # ➊ 
    print('start') 
    yield 'A' 
    print('continue') 
    yield 'B' 
    print('end.')
    
res1 = [x*3 for x in gen_AB()]  # ➋ 

start
continue
end.


In [28]:
for i in res1:  # ➌ 
    print('-->', i)

--> AAA
--> BBB


In [29]:
res2 = (x*3 for x in gen_AB())  # ➍
res2  # ➎

<generator object <genexpr> at 0x10b71bad0>

In [30]:
for i in res2:  # ➏
    print('-->', i)

start
--> AAA
continue
--> BBB
end.


➊ gen_AB 函数曾在第四节末尾出现过。

❷ 列表推导迫切地迭代 gen_AB() 函数生成的生成器对象产出的元素:'A' 和 'B'。注意，下面的输出是 start、continue 和 end.。

❸ 这个 for 循环迭代列表推导生成的 res1 列表。

❹ 把生成器表达式返回的值赋值给 res2。只需调用 gen_AB() 函数，虽然调用时会返回一个生成器，但是这里并不使用。

❺ res2 是一个生成器对象。

❻ 只有 for 循环迭代 res2 时，gen_AB 函数的定义体才会真正执行。for 循环每次迭代时会隐式调用 next(res2)，前进到 gen_AB 函数中的下一个 yield 语句。注意，gen_AB 函数的输出与 for 循环中 print 函数的输出夹杂在一起。

可以看出，生成器表达式会产出生成器，因此可以使用生成器表达式进一步减少 Sentence 类的代码。

In [31]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text): 
        self.text = text
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))

In [32]:
s = Sentence('"The time has come," the Walrus said,')
it = iter(s)
while True:
    try:
        print(next(it))
    except StopIteration:
        del it
        break

The
time
has
come
the
Walrus
said


通过上例可知，生成器表达式是创建生成器的简洁句法，这样无 需先定义函数再调用。不过，生成器函数灵活得多，可以使用多个语句实现复杂的逻辑，也可以作为协程使用。

生成器表达式是语法糖: 完全可以替换成生成器函数，不过有时使用生成器表达式更便利。

更多内容参见 *Fluent Python* 第 14 章。

## 7. 等差数列生成器

典型的迭代器模式作用很简单 —— 遍历数据结构。不过，即便不是从集合中获取元素，而是获取序列中即时生成的下一个值时，也用得到这种 基于方法的标准接口。例如，内置的 range 函数用于生成有穷整数等差数列，itertools.count函数用于生成无穷等差数列。

本节实现的 ArithmeticProgression 类可用于生成无穷等差数列。

In [33]:
class ArithmeticProgression:
    def __init__(self, begin, step, end=None):  # ➊ 
        self.begin = begin
        self.step = step
        self.end = end # None -> 无穷数列
        
    def __iter__(self):
        result = type(self.begin + self.step)(self.begin)  # ➋ 
        forever = self.end is None  # ➌
        index = 0
        while forever or result < self.end:  # ➍
            yield result  # ➎
            index += 1
            result = self.begin + self.step * index  # ➏

❶ \_\_init\_\_ 方法需要两个参数:begin 和 step。end 是可选的，如果值是 None，那么生成的是无穷数列。

❷ 这一行把 self.begin 赋值给 result，不过会先强制转换成前面的加法算式得到的类型。

❸ 为了提高可读性，我们创建了 forever 变量，如果 self.end 属 性的值是 None，那么 forever 的值是 True，因此生成的是无穷数列。

❹ 这个循环要么一直执行下去，要么当 result 大于或等于 self.end 时结束。如果循环退出了，那么这个函数也随之退出。 ❺ 生成当前的 result 值。

❻ 计算可能存在的下一个结果。这个值可能永远不会产出，因为 while 循环可能会终止。

In [34]:
# 生成有穷数列
ap = ArithmeticProgression(3, 10, 100)
list(ap)

[3, 13, 23, 33, 43, 53, 63, 73, 83, 93]

In [35]:
# 生成无穷数列
ap = iter(ArithmeticProgression(3, 10))
tmp = [next(ap)]
while tmp[-1] < 100:
    tmp.append(next(ap))
print(tmp[:-1])

[3, 13, 23, 33, 43, 53, 63, 73, 83, 93]


\* 使用生成器函数生成等差数列

In [36]:
def aritprog_gen(begin, step, end=None):
    result = type(begin + step)(begin)
    forever = end is None
    index = 0
    while forever or result < end:
        yield result
        index += 1
        result = begin + step * index  # 防止误差传递

In [37]:
# 生成有穷数列
ap = aritprog_gen(2, 10, 100)
list(ap)

[2, 12, 22, 32, 42, 52, 62, 72, 82, 92]

In [38]:
# 生成无穷数列
ap = aritprog_gen(2, 10)
tmp = [next(ap)]
while tmp[-1] < 100:
    tmp.append(next(ap))
print(tmp[:-1])

[2, 12, 22, 32, 42, 52, 62, 72, 82, 92]


\* 使用 itertools 模块生成等差数列

In [39]:
import itertools

def aritprog_gen(begin, step, end=None):
    first = type(begin + step)(begin)
    ap_gen = itertools.count(first, step)
    if end is not None:
        ap_gen = itertools.takewhile(lambda n: n < end, ap_gen)
    return ap_gen

注意，上例中的 aritprog_gen 不是生成器函数，因为定义体中没有 yield 关键字。但是它会返回一个生成器，因此它与其他生成器函数一样，也是生成器工厂函数。

In [40]:
# 生成有穷数列
ap = aritprog_gen(1, 10, 100)
list(ap)

[1, 11, 21, 31, 41, 51, 61, 71, 81, 91]

In [41]:
# 生成无穷数列
ap = aritprog_gen(1, 10)
tmp = [next(ap)]
while tmp[-1] < 100:
    tmp.append(next(ap))
print(tmp[:-1])

[1, 11, 21, 31, 41, 51, 61, 71, 81, 91]


## 8. 标准库中的生成器函数

标准库提供了很多生成器，有用于逐行迭代纯文本文件的对象。

### 8.1 用于过滤的生成器函数

In [42]:
def vowel(c):
    return c.lower() in 'abcde'

In [43]:
# 输出 vowel 的计算结果为真的元素
list(filter(vowel, 'AardvarK'))

['A', 'a', 'd', 'a']

In [44]:
# 输出 vowel 的计算结果为假的元素
import itertools
list(itertools.filterfalse(vowel, 'AardvarK'))

['r', 'v', 'r', 'K']

In [45]:
# 输出 vowel 计算结果为真的元素，如遇 False 不再继续检查
list(itertools.takewhile(vowel, 'AardvarK'))

['A', 'a']

In [46]:
# 跳过 vowel 计算结果为真的元素（和 while 逻辑相似，如遇 False 将不再进一步检查），然后输出剩余的元素
list(itertools.dropwhile(vowel, 'AardvarK'))
# list(itertools.dropwhile(vowel, 'fAardvarK'))

['r', 'd', 'v', 'a', 'r', 'K']

In [47]:
# compress(it, selector_it)
# 并行处理两个可迭代的对象; 如果 selector_it 中的元素是真值，产出 it 中对应的元素
list(itertools.compress('Aardvark', (1,0,1,1,0,1)))

['A', 'r', 'd', 'a']

In [48]:
# islice(it, start[, stop, step=1])
# 类似切片
list(itertools.islice('Aardvark', 4))

['A', 'a', 'r', 'd']

In [49]:
list(itertools.islice('Aardvark', 4, 7))

['v', 'a', 'r']

In [50]:
list(itertools.islice('Aardvark', 1, 7, 2))

['a', 'd', 'a']

### 8.2 用于映射的生成器函数

\* 演示 itertools.accumulate 生成器函数

In [51]:
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]

In [52]:
# 计算总和
import itertools
list(itertools.accumulate(sample))

[5, 9, 11, 19, 26, 32, 35, 35, 44, 45]

In [53]:
# 计算最小值
list(itertools.accumulate(sample, min))

[5, 4, 2, 2, 2, 2, 2, 0, 0, 0]

In [54]:
# 计算最大值
list(itertools.accumulate(sample, max))

[5, 5, 5, 8, 8, 8, 8, 8, 9, 9]

In [55]:
# 计算乘积
import operator
list(itertools.accumulate(sample, operator.mul))

[5, 20, 40, 320, 2240, 13440, 40320, 0, 0, 0]

In [56]:
# 从 1! 到 10!，计算各个数的阶乘
list(itertools.accumulate(range(1, 11), operator.mul)) 

[1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

\* 演示用于映射的生成器函数。

In [57]:
# 从 3 开始，为单词中的字母编号。
list(enumerate('albatroz', 3))

[(3, 'a'),
 (4, 'l'),
 (5, 'b'),
 (6, 'a'),
 (7, 't'),
 (8, 'r'),
 (9, 'o'),
 (10, 'z')]

In [58]:
# 从 0 到 10，计算各个整数的平方。
import operator
list(map(operator.mul, range(11), range(11)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [59]:
# 计算两个可迭代对象中对应位置上的两个元素之积，元素最少的那个可迭代对象到头后就停止。
list(map(operator.mul, range(11), [2, 4, 8]))

[0, 4, 16]

In [60]:
# 作用等同于内置的 zip 函数。
list(map(lambda a, b: (a, b), range(11), [2, 4, 8]))

[(0, 2), (1, 4), (2, 8)]

In [61]:
# 从 1 开始，根据字母所在的位置，把字母重复相应的次数。
import itertools
list(itertools.starmap(operator.mul, enumerate('albatroz', 1)))

['a', 'll', 'bbb', 'aaaa', 'ttttt', 'rrrrrr', 'ooooooo', 'zzzzzzzz']

In [62]:
# 计算平均值。
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]
list(itertools.starmap(lambda a, b: b/a,
                      enumerate(itertools.accumulate(sample), 1)))
# list(enumerate(itertools.accumulate(sample), 1))

[5.0,
 4.5,
 3.6666666666666665,
 4.75,
 5.2,
 5.333333333333333,
 5.0,
 4.375,
 4.888888888888889,
 4.5]

### 8.3 用于合并的生成器函数

In [63]:
# 调用 chain 函数时通常传入两个或更多个可迭代对象
list(itertools.chain('ABC', range(2)))

['A', 'B', 'C', 0, 1]

In [64]:
# 如果只传入一个可迭代的对象，那么 chain 函数没什么用
list(itertools.chain(enumerate('ABC')))

[(0, 'A'), (1, 'B'), (2, 'C')]

In [65]:
# 但是 chain.from_iterable 函数从可迭代的对象中获取每个元素，然后按顺序把元素连接起来
# 前提是各个元素本身也是可迭代的对象
list(itertools.chain.from_iterable(enumerate('ABC')))

[0, 'A', 1, 'B', 2, 'C']

In [66]:
# zip 常用于把两个可迭代的对象合并成一系列由两个元素组成的元组
list(zip('ABC', range(5)))

[('A', 0), ('B', 1), ('C', 2)]

In [67]:
# zip 可以并行处理任意数量个可迭代的对象
# 不过只要有一个可迭代的对象到头了，生成器就停止
list(zip('ABC', range(5), [10, 20, 30, 40]))

[('A', 0, 10), ('B', 1, 20), ('C', 2, 30)]

In [68]:
# itertools.zip_longest 函数的作用与 zip 类似
# 不过输入的所有可迭代对象都会处理到头，如果需要会填充 None
list(itertools.zip_longest('ABC', range(5)))

[('A', 0), ('B', 1), ('C', 2), (None, 3), (None, 4)]

In [69]:
# fillvalue 关键字参数用于指定填充的值。
list(itertools.zip_longest('ABC', range(5), fillvalue='?'))

[('A', 0), ('B', 1), ('C', 2), ('?', 3), ('?', 4)]

\* 演示 itertools.product 生成器函数

itertools.product 生成器是计算笛卡儿积的惰性方式。

In [70]:
# 三个字符的字符串与两个整数的值域得到的笛卡儿积是六个元组 (因为3 * 2等于6)。
list(itertools.product('ABC', range(2)))

[('A', 0), ('A', 1), ('B', 0), ('B', 1), ('C', 0), ('C', 1)]

In [71]:
# 两张牌('AK')与四种花色得到的笛卡儿积是八个元组。
suits = 'spades hearts diamonds clubs'.split()
list(itertools.product('AK', suits))

[('A', 'spades'),
 ('A', 'hearts'),
 ('A', 'diamonds'),
 ('A', 'clubs'),
 ('K', 'spades'),
 ('K', 'hearts'),
 ('K', 'diamonds'),
 ('K', 'clubs')]

In [72]:
# 如果传入一个可迭代的对象，product 函数产出的是一系列只有一个元素的元组，不是特别有用。
list(itertools.product('ABC'))

[('A',), ('B',), ('C',)]

In [73]:
# repeat=N 关键字参数告诉 product 函数重复 N 次处理输入的各个可迭代对象。
list(itertools.product('ABC', repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

In [74]:
list(itertools.product(range(2), repeat=3))

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [75]:
rows = itertools.product('AB', range(2), repeat=2)
for row in rows: print(row)

('A', 0, 'A', 0)
('A', 0, 'A', 1)
('A', 0, 'B', 0)
('A', 0, 'B', 1)
('A', 1, 'A', 0)
('A', 1, 'A', 1)
('A', 1, 'B', 0)
('A', 1, 'B', 1)
('B', 0, 'A', 0)
('B', 0, 'A', 1)
('B', 0, 'B', 0)
('B', 0, 'B', 1)
('B', 1, 'A', 0)
('B', 1, 'A', 1)
('B', 1, 'B', 0)
('B', 1, 'B', 1)


\* 演示 count、repeat 和 cycle 的用法

In [76]:
# 使用 count 函数构建 ct 生成器。
ct = itertools.count()
# 不能使用 ct 构建列表，因为 ct 是无穷的，所以获取 3 个元素
next(ct), next(ct), next(ct) 

(0, 1, 2)

In [77]:
# 如果使用 islice 或 takewhile 函数做了限制，可以从 count 生成器中构建列表。
list(itertools.islice(itertools.count(1, .3), 3))

[1, 1.3, 1.6]

In [78]:
# 使用 'ABC' 构建一个 cycle 生成器，然后获取第一个元素
cy = itertools.cycle('ABC')
next(cy)

'A'

In [79]:
# 只有受到 islice 函数的限制，才能构建列表; 这里获取接下来的 7 个元素。
list(itertools.islice(cy, 7))

['B', 'C', 'A', 'B', 'C', 'A', 'B']

In [80]:
# 构建一个 repeat 生成器，始终产出数字 7。
rp = itertools.repeat(7)
next(rp), next(rp), next(rp)

(7, 7, 7)

In [81]:
# 传入 times 参数可以限制 repeat 生成器生成的元素数量: 这里会生成4次数字8。
list(itertools.repeat(8, 4))

[8, 8, 8, 8]

In [82]:
# repeat 函数的常见用途: 为 map 函数提供固定参数，这里提供的是乘数 5。
list(map(operator.mul, range(11), itertools.repeat(5)))

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

在 [itertools 模块的文档](https://docs.python.org/3/library/itertools.html)中，combinations、combinatio 和 permutations 生成器函数，连同 product 函数，称为组合学生成器(combinatoric generator)。itertools.product函数和其余的组合学函数有紧密的联系。

### 8.4 组合学生成器函数

In [83]:
#  'ABC' 中每两个元素(len()==2)的各种组合; 
# 在生成的元组中，元素的顺序无关紧要(可以视作集合)。
list(itertools.combinations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'C')]

In [84]:
# 'ABC' 中每两个元素(len()==2)的各种组合，包括相同元素的组合。
list(itertools.combinations_with_replacement('ABC', 2))

[('A', 'A'), ('A', 'B'), ('A', 'C'), ('B', 'B'), ('B', 'C'), ('C', 'C')]

In [85]:
# 'ABC' 中每两个元素(len()==2)的各种排列;在生成的元组 中，元素的顺序有重要意义。
list(itertools.permutations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'A'), ('B', 'C'), ('C', 'A'), ('C', 'B')]

In [86]:
# 'ABC' 和 'ABC'(repeat=2 的效果)的笛卡儿积。
list(itertools.product('ABC', repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

本节要讲的最后一组生成器函数用于产出输入的可迭代对象中的全部元素，不过会以某种方式重新排列。其中有两个函数会返回多个生成器， 分别是 itertools.groupby 和 itertools.tee。这一组里的另一个生成器函数，内置的 reversed 函数，是本节所述的函数中唯一一个不接受可迭代的对象，而只接受序列为参数的函数。这在情理之中，因为 reversed 函数从后向前产出元素，而只有序列的长度已知时才能工作。不过，这个函数会按需产出各个元素，因此无需创建反转的副本。

### 8.5 用于重新排列元素的生成器函数

\* 演示 itertools.groupby 的用法

注意，itertools.groupby 假定输入的可迭代对象要使用分组标准排序; 即使不排序，至少也要使用指定的标准分组各个元素。

In [87]:
# groupby 函数产出 (key, group_generator) 这种形式的元组。
list(itertools.groupby('LLLLAAGGG'))

[('L', <itertools._grouper at 0x10b777b10>),
 ('A', <itertools._grouper at 0x10b777c50>),
 ('G', <itertools._grouper at 0x10b777ad0>)]

In [88]:
# 处理 groupby 函数返回的生成器要嵌套迭代: 这里在外层使用 for 循环，内层使用列表推导。
for char, group in itertools.groupby('LLLLAAAGG'):
    print(char, '->', list(group))

L -> ['L', 'L', 'L', 'L']
A -> ['A', 'A', 'A']
G -> ['G', 'G']


In [89]:
# 为了使用 groupby 函数，要排序输入; 这里按照单词的长度排序。
animals = ['duck', 'eagle', 'rat', 'giraffe', 'bear',
           'bat', 'dolphin', 'shark', 'lion']
# sorted(animals, key=len, reverse=True)
animals.sort(key=len)
animals

['rat', 'bat', 'duck', 'bear', 'lion', 'eagle', 'shark', 'giraffe', 'dolphin']

In [90]:
# 再次遍历 key 和 group 值对，把 key 显示出来，并把 group 扩展成列表。
for length, group in itertools.groupby(animals, len):
    print(length, '->', list(group))

3 -> ['rat', 'bat']
4 -> ['duck', 'bear', 'lion']
5 -> ['eagle', 'shark']
7 -> ['giraffe', 'dolphin']


In [91]:
# 这里使用 reverse 生成器从右向左迭代 animals。
for length, group in itertools.groupby(reversed(animals), len):
    print(length, '->', list(group))

7 -> ['dolphin', 'giraffe']
5 -> ['shark', 'eagle']
4 -> ['lion', 'bear', 'duck']
3 -> ['bat', 'rat']


\* 演示 itertools.tee 的用法

itertools.tee 函数产出多个生成器，每个生成器都可以产出输入的各个元素.

In [92]:
list(itertools.tee('ABC'))

[<itertools._tee at 0x10b774e10>, <itertools._tee at 0x10b774c30>]

In [93]:
g1, g2 = itertools.tee('ABC')
next(g1)

'A'

In [94]:
next(g2), next(g2)

('A', 'B')

In [95]:
list(g1)

['B', 'C']

In [96]:
list(g2)

['C']

In [97]:
list(zip(*itertools.tee('ABC')))

[('A', 'A'), ('B', 'B'), ('C', 'C')]

## 9. Python 3.3 中新出现的句法: yield from

> yield from 的作用是把不同的生成器结合在一起使用。

如果生成器函数需要产出另一个生成器生成的值，传统的解决方法是使用嵌套的 for 循环。

In [98]:
def chain(*iterables):
    for it in iterables:
        for i in it:
            yield i

In [99]:
s = 'ABC'
t = tuple(range(3))
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

chain 生成器函数把操作依次交给接收到的各个可迭代对象处理。为此，[PEP 380 — Syntax for Delegating to a Subgenerator](https://www.python.org/dev/peps/pep-0380/)引入了一个新句法，如下述控制台中的代码清单所示:

In [100]:
def chain(*iterables):
    for i in iterables:
        yield from i

In [101]:
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

可以看出，`yield from i` 完全代替了内层的 for 循环。在这个示例 中使用 `yield from` 是对的，而且代码读起来更顺畅，不过感觉更像是语法糖。除了代替循环之外，`yield from` 还会创建通道，把内层生成器直接与外层生成器的客户端联系起来。把生成器当成协程使用 时，这个通道特别重要，不仅能为客户端代码生成值，还能使用客户端代码提供的值。

## 10. 可迭代的归约函数

把接受一个可迭代的对象，然后返回单个结果的函数叫“归约”函数、“合拢”函数或“累加”函数。

In [102]:
# all(it)
# it 中的所有元素都为真值时返回 True，否则返回 False; 
all([1, 2, 3])

True

In [103]:
all([1, 0, 3])

False

In [104]:
# all([]) 返回 True
all([])

True

In [105]:
# 只要 it 中有元素为真值就返回 True，否则返回 False
any([1, 0, 3]), any([1, 2, 3])

(True, True)

In [106]:
any([0, 0, 0])

False

In [107]:
# any([]) 返回 False
any([])

False

any 会短路，即一旦确定了结果就停止使用迭代器。

In [108]:
g = (n for n in [0, 0.0, 7, 8])
any(g)

True

In [109]:
next(g)

8

In [110]:
import functools
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]
functools.reduce(lambda a, b: a+b, sample)

45

## 11. 深入分析iter函数

如前所述，在 Python 中迭代对象 x 时会调用 iter(x)。可是，iter 函数还有一个鲜为人知的用法: 传入两个参数，使用常规的函数或任何可调用的对象创建迭代器。这样使用时，第一个参数必须 是可调用的对象，用于不断调用(没有参数)，产出各个值; 第二个值是哨符，这是个标记值，当可调用的对象返回这个值时，触发迭代器抛出 StopIteration 异常，而不产出哨符。

\* 掷骰子

In [111]:
from random import randint
def d6():
    return randint(1, 6)

d6_iter = iter(d6, 1)
d6_iter

In [112]:
for roll in d6_iter:
    print(roll)

3
3


注意，这里的 iter 函数返回一个 callable_iterator 对象。示例中的 for 循环可能运行特别长的时间，不过肯定不会打印 1，因为 1 是哨符。与常规的迭代器一样，这个示例中的 d6_iter 对象一旦耗尽就没用了。如果想重新开始，必须再次调用 iter(...)，重新构建迭代器。

\* 输出从 3 开始的 step 为 1 的等差数列，遇到哨符时停止

In [119]:
import itertools

class Count:
    def __init__(self, begin, step):
        self.ic = itertools.count(begin, step)
        
    def next_num(self):
        return next(self.ic)

In [120]:
count = Count(3,1)
nn = iter(count.next_num, 10)
nn

In [121]:
while True:
    try:
        print(next(nn))
    except StopIteration:
        del nn
        break

3
4
5
6
7
8
9


[内置函数 iter 的文档](https://docs.python.org/3/library/functions.html#iter)中有个实用的例子。这段代码逐行读取文件，直到遇到空行或者到达文件末尾为止:

```python
with open('mydata.txt') as fp:
    for line in iter(fp.readline, '\n'):
        process_line(line)
```